## Workflow to update a TX County data layer (shapefile) with the latest COVID-19 cases and load it to ArcGIS Online

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from arcgis.gis import GIS

In [ ]:
'''TX DSHS quit putting thier data on a table in the link below. Instead, they are using AGOL.
Im keep this here as a reference.'''

# from urllib.request import urlopen
# from bs4 import BeautifulSoup
# import ssl

# # scrape Texas Department of State Health Services
# ssl._create_default_https_context = ssl._create_unverified_context # avoid cert error 
# url = 'https://www.dshs.state.tx.us/news/updates.shtm#coronavirus'
# html = urlopen(url)
# soup = BeautifulSoup(html, 'lxml')

# # get count of latest COVID-19
# list_for_df = []
# for tr in soup.find_all('tr'):
#     td = tr.find_all('td')
#     row = [tr.text for tr in td]
#     list_for_df.append(row)

# # create df with counts, use filter() to remove empty list
# count_df = pd.DataFrame(list(filter(None,list_for_df)), columns=['CNTY_NM', 'COUNT'])

In [ ]:
# values for GIS() from a file 
f = open('agol.csv')
username = str(f.readline())[:-1]
password = str(f.readline())[:-1]
gis = GIS("https://www.arcgis.com", username, password)

In [ ]:
# TX DSHS COVID-19 layer
covid_19_item_id = gis.content.get('bc83058386d2434ca8cf90b26dc6b580')
covid_19_item_id

In [ ]:
# grab the covid-19 data layer and turn it into a df
covid_19_item_df = pd.DataFrame.spatial.from_layer(item_id.layers[0])
del covid_19_item_df['LastUpdate'] # remove this b/c shapefiles don't like it
covid_19_item_df.info()

In [ ]:
# read in a county shapefile using GeoPandas
county_gdf_raw = gpd.GeoDataFrame(gpd.read_file(r'data/origs/Texas_County_Boundaries.shp'))
county_gdf = county_gdf_raw[['OBJECTID','CNTY_NM','geometry']]
county_gdf.head()

In [ ]:
# merge df's to add COVID19 count to geometry
merged_df = county_gdf.merge(covid_19_item_df, left_on='CNTY_NM', right_on='County', how='left')

In [ ]:
import shutil
from datetime import date
today = date.today()

In [ ]:
# write to file
merged_df.to_file('data/import/tx_counties_covid19_{}.shp'.format(today))

# dosen't create a .prj file, so use the one from the original data
shutil.copyfile('data/origs/Texas_County_Boundaries.prj', 'data/import/tx_counties_covid19_{}.prj'.format(today))


In [ ]:
# zip shapefile
shutil.make_archive('data/import/tx_counties_covid19_{}'.format(today), 'zip', 'data/import')

In [ ]:
# get the old id and delete it
# old_shp_id = gis.content.get('ab59470b45ef42e3ae10a1d825c05f1b')
# old_shp_id.delete()

In [56]:
# add data to AGOL

# create properties
tx_county_covid19_properties = {
    'title': 'Shapefile_TX_Counties_COVID19_Count_{}'.format(today),
    'snippet': 'TX DSHS provides a data of COVID-19 cases by county. Here is a shapefile of that service. https://txdshs.maps.arcgis.com/apps/opsdashboard/index.html#/ed483ecd702b4298ab01e8b9cafc8b83 ',
    'description': '''This is a shapefile of TX COVID-19 data. The data source is the Texas Department of State Health Services. DSHS updates their counts everyday at noon (Central). Expect this layer to be updated with the latest counts not long after DSHS release their numbers.\n


Data source: Source: https://services5.arcgis.com/ACaLB9ifngzawspq/ArcGIS/rest/services/COVID19County_ViewLayer/FeatureServer

Want to see how this was created? https://github.com/dandresen/arcgis-python-api/blob/master/COVID-19%20TX%20Counties.ipynb''',
    'tags': 'tx, open data, COVID-19, COVID, TEXAS, shapefile',
    'type': 'Shapefile',
    'licenseInfo': '''This is a NON-OFFICIAL resource. The author does not assume any responsibly for errors in the data. The only data source used for COVID-19 counts is from the Texas Department of State Health Services website. Please visit their site to understand more about how the data was collected.'''
}

# get the file location and add to AGOL 
data_file_location = r'data/import/tx_counties_covid19_{}.zip'.format(today)
tx_county_covid19_file = gis.content.add(tx_county_covid19_properties, data=data_file_location)
tx_county_covid19_file.share(everyone=True)

{'results': [{'itemId': '0e06a11a72f94870bb398846ec047023',
   'success': True,
   'notSharedWith': []}]}

In [57]:
# publish the shapefile 
tx_county_covid19_publish = tx_county_covid19_file.publish()
tx_county_covid19_publish.share(everyone=True)

{'results': [{'itemId': '2b84ebf268e540969de866beafd590b6',
   'success': True,
   'notSharedWith': []}]}